This is part of the CTR prediction project. Here, I use wide and deep, DeepFM, xDeepFM from DeepCTR. 

In [ ]:
!pip install -U deepctr-torch
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('drive/My Drive/20210401/CTR/filtered_train.csv')

data['hr'] = data['hour'].astype(str).str.slice(6, 8).astype(int)
data = data.iloc[: , 1:]
data = data.drop('id', axis = 1)
# maxBins = 70
# categorical = [item[0] for item in strColsCount if item[1] <= maxBins]
# categorical += [item[0] for item in intColsCount if item[1] <= maxBins] 
print(data.columns)

#print(data.loc[1,'hr'])
# sparse_features = ['C' + str(i) for i in range(1, 27)]
# dense_features = ['I' + str(i) for i in range(1, 14)]

# data[sparse_features] = data[sparse_features].fillna('-1', )
# data[dense_features] = data[dense_features].fillna(0, )
# target = ['label']

Index(['click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'hr'],
      dtype='object')


In [ ]:
data.shape

(404290, 24)

In [ ]:
data.dtypes

click                int64
hour                 int64
C1                   int64
banner_pos           int64
site_id             object
site_domain         object
site_category       object
app_id              object
app_domain          object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type          int64
device_conn_type     int64
C14                  int64
C15                  int64
C16                  int64
C17                  int64
C18                  int64
C19                  int64
C20                  int64
C21                  int64
hr                   int64
dtype: object

In [ ]:
data.nunique()

click                    2
hour                   240
C1                       7
banner_pos               7
site_id               2225
site_domain           2188
site_category           22
app_id                2241
app_domain             143
app_category            27
device_id            64742
device_ip           261706
device_model          4380
device_type              4
device_conn_type         4
C14                   2088
C15                      8
C16                      9
C17                    411
C18                      4
C19                     65
C20                    161
C21                     60
hr                      24
dtype: int64

In [ ]:
data['click'].value_counts()

0    335635
1     68655
Name: click, dtype: int64

In [ ]:
# obj_features = list(data.select_dtypes(['object']).columns)
# int_features = list(data.select_dtypes(['int64']).columns)
# int_features.remove('click')

# data[obj_features] = data[obj_features].fillna('-1', )
# data[int_features] = data[int_features].fillna(0, )

# for feat in obj_features:
#     lbe = LabelEncoder()
#     data[feat] = lbe.fit_transform(data[feat])

# mms = MinMaxScaler(feature_range=(0, 1))
# data[int_features] = mms.fit_transform(data[int_features])


# strCols = [item[0] for item in data.dtypes if item[1] == 'string']
# intCols = [item[0] for item in data.dtypes if item[1] == 'int64']
# for idx,item in enumerate(data.dtypes):
#   print(idx, item)

# sparse_features = ['C1','banner_pos','site_category','app_category','device_type','device_conn_type','C15','C16','C18','C19','C21','hr']
# dense_features = ['site_id','site_domain','app_id','app_domain','device_id','device_ip','device_model','C14','C17','C20']

# data[sparse_features] = data[sparse_features].fillna('-1', )
# data[dense_features] = data[dense_features].fillna(0, )
# target = ['click']

# #features = list(data.columns)
# sparse_features = list(data.select_dtypes(['object']).columns)
# dense_features = list(data.select_dtypes(['int64']).columns)
# dense_features.remove('click')
# sparse_features.remove('device_id')
# sparse_features.remove('device_ip')

# # data.rename(columns={'click':'label'}, inplace=True)

# data[sparse_features] = data[sparse_features].fillna('-1', )
# data[dense_features] = data[dense_features].fillna(0, )
# target = ['click']

# print(sparse_features)
# print(dense_features)

# for column_name in data.columns:
#     column = data[column_name]
#     # Get the count of Zeros in column 
#     count = (column == 0).sum()
#     print('Count of zeros in column ', column_name, ' is : ', count/data.shape[0]*100)

# #data_count = data.nunique().reset_index() 
# dense_features = data.loc[:, data.nunique() > 70].columns.tolist()
# dense_features.remove('device_id')
# dense_features.remove('device_ip')

# print(dense_features)

In [ ]:

# features = list(data.select_dtypes(['object']).columns)
# feature_max_idx = {}
# for feature in features:
#     lbe = LabelEncoder()
#     data[feature] = lbe.fit_transform(data[feature]) + 1
#     feature_max_idx[feature] = data[feature].max() + 1

# feature_max_idx

In [ ]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
sparse_features = list(data.select_dtypes(['object']).columns)
dense_features = list(data.select_dtypes(['int64']).columns)
dense_features.remove('click')
sparse_features.remove('device_id')
sparse_features.remove('device_ip')

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['click']

for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [ ]:
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                          for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                          for feat in dense_features]
#print(fixlen_feature_columns)
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

# fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
#                           for feat in sparse_features] + [DenseFeat(feat, 1, )
#                                                           for feat in dense_features]
# dnn_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_features]
# linear_feature_columns = [SparseFeat(feat, data[feat].nunique()) for feat in sparse_features]

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [ ]:
# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2, random_state=2022)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

#print(train_model_input[0,:])
# print(test[target].value_counts())

In [ ]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                task='binary',
                l2_reg_embedding=1e-5, device=device)

model.compile("adagrad", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"], )

history = model.fit(train_model_input, train[target].values, epochs=10, verbose=2,validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda ready...
cuda:0
Train on 258745 samples, validate on 64687 samples, 1011 steps per epoch
Epoch 1/10
31s - loss:  0.4115 - binary_crossentropy:  0.4115 - auc:  0.7201 - val_binary_crossentropy:  0.4053 - val_auc:  0.7273
Epoch 2/10
14s - loss:  0.4010 - binary_crossentropy:  0.4010 - auc:  0.7420 - val_binary_crossentropy:  0.4048 - val_auc:  0.7286
Epoch 3/10
15s - loss:  0.3979 - binary_crossentropy:  0.3979 - auc:  0.7482 - val_binary_crossentropy:  0.4049 - val_auc:  0.7286
Epoch 4/10
15s - loss:  0.3959 - binary_crossentropy:  0.3959 - auc:  0.7520 - val_binary_crossentropy:  0.4058 - val_auc:  0.7276
Epoch 5/10
15s - loss:  0.3943 - binary_crossentropy:  0.3943 - auc:  0.7544 - val_binary_crossentropy:  0.4068 - val_auc:  0.7274
Epoch 6/10
14s - loss:  0.3932 - binary_crossentropy:  0.3932 - auc:  0.7565 - val_binary_crossentropy:  0.4071 - val_auc:  0.7257
Epoch 7/10
15s - loss:  0.3922 - binary_crossentropy:  0.3922 - auc:  0.7578 - val_binary_crossentropy:  0.4074 - val_au

In [ ]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = xDeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                task='binary',
                l2_reg_embedding=1e-5, device=device)

model.compile("adagrad", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"], )

history = model.fit(train_model_input, train[target].values, epochs=10, verbose=2,validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda ready...
cuda:0
Train on 258745 samples, validate on 64687 samples, 1011 steps per epoch
Epoch 1/10
21s - loss:  0.4103 - binary_crossentropy:  0.4103 - auc:  0.7218 - val_binary_crossentropy:  0.4048 - val_auc:  0.7284
Epoch 2/10
21s - loss:  0.4002 - binary_crossentropy:  0.4002 - auc:  0.7437 - val_binary_crossentropy:  0.4049 - val_auc:  0.7295
Epoch 3/10
21s - loss:  0.3968 - binary_crossentropy:  0.3968 - auc:  0.7500 - val_binary_crossentropy:  0.4053 - val_auc:  0.7284
Epoch 4/10
20s - loss:  0.3947 - binary_crossentropy:  0.3947 - auc:  0.7539 - val_binary_crossentropy:  0.4059 - val_auc:  0.7267
Epoch 5/10
21s - loss:  0.3930 - binary_crossentropy:  0.3931 - auc:  0.7561 - val_binary_crossentropy:  0.4072 - val_auc:  0.7255
Epoch 6/10
21s - loss:  0.3918 - binary_crossentropy:  0.3918 - auc:  0.7578 - val_binary_crossentropy:  0.4078 - val_auc:  0.7250
Epoch 7/10
21s - loss:  0.3907 - binary_crossentropy:  0.3907 - auc:  0.7590 - val_binary_crossentropy:  0.4085 - val_au

In [ ]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = WDL(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                task='binary',
                l2_reg_embedding=1e-5, device=device)

model.compile("adagrad", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"], )

history = model.fit(train_model_input, train[target].values, epochs=10, verbose=2,validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda ready...
cuda:0
Train on 258745 samples, validate on 64687 samples, 1011 steps per epoch
Epoch 1/10
14s - loss:  0.4115 - binary_crossentropy:  0.4115 - auc:  0.7201 - val_binary_crossentropy:  0.4052 - val_auc:  0.7276
Epoch 2/10
14s - loss:  0.4011 - binary_crossentropy:  0.4010 - auc:  0.7421 - val_binary_crossentropy:  0.4047 - val_auc:  0.7290
Epoch 3/10
14s - loss:  0.3979 - binary_crossentropy:  0.3979 - auc:  0.7483 - val_binary_crossentropy:  0.4048 - val_auc:  0.7288
Epoch 4/10
14s - loss:  0.3959 - binary_crossentropy:  0.3959 - auc:  0.7521 - val_binary_crossentropy:  0.4058 - val_auc:  0.7278
Epoch 5/10
14s - loss:  0.3943 - binary_crossentropy:  0.3943 - auc:  0.7545 - val_binary_crossentropy:  0.4067 - val_auc:  0.7275
Epoch 6/10
14s - loss:  0.3932 - binary_crossentropy:  0.3932 - auc:  0.7565 - val_binary_crossentropy:  0.4070 - val_auc:  0.7258
Epoch 7/10
14s - loss:  0.3923 - binary_crossentropy:  0.3923 - auc:  0.7577 - val_binary_crossentropy:  0.4073 - val_au

In [ ]:
# data1 = pd.read_csv('drive/My Drive/20210401/CTR/train_full.csv')
# print(data1.iloc[0,:])
# print(data1.columns)